In [5]:
from konlpy.tag import Okt

def extract_keywords(search_query):
    okt = Okt()
    keywords = okt.nouns(search_query)
    return keywords

In [6]:
"""
import mysql.connector

def recommend_stickers(search_query):
    # 데이터베이스에 연결한다.
    connection = mysql.connector.connect(
        host="127.0.0.1",
        user="jspbook",
        password="passwd",
        database="every"
    )

    # 검색어와 연관된 키워드를 추출한다.
    keywords = extract_keywords(search_query)

    # 각 키워드와 가장 유사한 스티커를 찾아서 추천한다.
    recommendations = []
    for keyword in keywords:
        similar_stickers = model.wv.most_similar(keyword)
        for sticker in similar_stickers:
            recommendations.append(sticker[0])

    # 추천 결과에서 중복된 스티커를 제거한다.
    recommendations = list(set(recommendations))

    # 데이터베이스에서 추천된 스티커 정보를 가져온다.
    cursor = connection.cursor()
    query = "SELECT * FROM tb_stickers WHERE name IN %s"
    values = (recommendations,)
    cursor.execute(query, values)
    result = cursor.fetchall()

    # 데이터베이스 연결을 종료한다.
    cursor.close()
    connection.close()

    # 추천된 스티커를 반환한다.
    return result
"""


import mysql.connector

def recommend_stickers(search_query):
    # 데이터베이스에 연결한다.
    connection = mysql.connector.connect(
        host="127.0.0.1",
        user="jspbook",
        password="passwd",
        database="every"
    )

    # 검색어와 연관된 키워드를 추출한다.
    keywords = extract_keywords(search_query)

    # 각 키워드와 가장 유사한 스티커를 찾아서 추천한다.
    recommendations = []
    for keyword in keywords:
        similar_stickers = model.wv.most_similar(keyword)
        for sticker in similar_stickers:
            recommendations.append(sticker[0])

    # 추천된 스티커를 가져올 때, 데이터베이스에서 리스트 형태로 전달하지 않도록 수정한다.
    recommendations = list(set(recommendations))
    recommendations_tuple = tuple(recommendations)

    # 데이터베이스에서 추천된 스티커 정보를 가져온다.
    cursor = connection.cursor()
    query = "SELECT * FROM tb_stickers WHERE name IN %s"
    values = (recommendations_tuple,)
    cursor.execute(query, values)
    result = cursor.fetchall()

    # 데이터베이스 연결을 종료한다.
    cursor.close()
    connection.close()

    # 추천된 스티커를 반환한다.
    return result




In [7]:
res = recommend_stickers('c')

print(res)

ProgrammingError: Failed processing format-parameters; Python 'tuple' cannot be converted to a MySQL type

In [1]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\juyz0\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\juyz0\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\juyz0\AppData\Roaming\nltk_data...


True

In [6]:
import mysql.connector
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

def preprocess(text):
    # 소문자로 변환합니다.
    text = text.lower()

    # 토큰화합니다.
    tokens = word_tokenize(text)

    # 불용어를 제거합니다.
    stop_words = set(stopwords.words("english"))
    tokens = [token for token in tokens if token not in stop_words]

    # 표제어 추출을 수행합니다.
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # 토큰들을 다시 결합하여 하나의 문자열로 만듭니다.
    text = " ".join(tokens)
    return text

def search_file_path(hostname, username, password, database, search_word):
    # 데이터베이스에 연결합니다.
    cnx = mysql.connector.connect(host=hostname, user=username, password=password, database=database)

    # SQL 쿼리를 작성합니다. 모든 레코드를 읽어와 토큰화한 후 TF-IDF 벡터로 변환합니다.
    query = f"SELECT id, name, file_path FROM stickers"
    df = pd.read_sql(query, cnx)
    df['name'] = df['name'].apply(preprocess)
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(df['name'])

    # 검색어를 토큰화하여 TF-IDF 벡터로 변환합니다.
    search_word = preprocess(search_word)
    search_vector = vectorizer.transform([search_word])

    # 유사도를 계산합니다.
    scores = X.dot(search_vector.T).toarray().flatten()
    idxmax = scores.argmax()
    print(scores)
    result = df.loc[idxmax, 'file_path']

    # 연결을 종료합니다.
    cnx.close()

    # 결과를 반환합니다.
    return result

In [7]:
search_file_path('127.0.0.1', 'jspbook', 'passwd', 'every', 'fruit')

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


'temp1'

In [6]:
import nltk
from nltk.corpus import wordnet

# 두 단어의 유사도를 계산하는 함수
def word_similarity(word1, word2):
    # 각 단어의 동의어와 반의어를 가져옵니다.
    synonyms1 = wordnet.synsets(word1)
    synonyms2 = wordnet.synsets(word2)
    
    # 두 단어의 유사도를 계산합니다.
    max_similarity = 0
    for syn1 in synonyms1:
        for syn2 in synonyms2:
            similarity = syn1.wup_similarity(syn2)
            if similarity is not None and similarity > max_similarity:
                max_similarity = similarity
    
    return max_similarity

# 두 단어의 유사도를 출력합니다.
word1 = "banana"
word2 = "apple"
similarity = word_similarity(word1, word2)
print(f"The similarity between {word1} and {word2} is {similarity}")

The similarity between banana and apple is 0.8181818181818182


In [2]:
import pandas as pd

# DataFrame 생성
df = pd.DataFrame({'id': [1, 2, 3], 'tag': ['dog', 'cat', 'apple']})

# 결과 출력
df

,id,tag
0,1,dog
1,2,cat
2,3,apple


In [8]:
import pandas as pd
from nltk.corpus import wordnet

# DataFrame 생성
df = pd.DataFrame({'id': [1, 2, 3], 'tag': ['dog', 'cat', 'apple']})

# 비교할 단어
food = 'banana'

# score 컬럼 추가
df['score'] = 0

# 단어 유사도 계산
for i, row in df.iterrows():
    tag = row['tag']
    synsets_tag = wordnet.synsets(tag)
    synsets_food = wordnet.synsets(food)
    if synsets_tag and synsets_food:
        score = synsets_tag[0].wup_similarity(synsets_food[0])
        df.at[i, 'score'] = score

# 결과 출력
print(df)

   id    tag  score
0   1    dog      0
1   2    cat      0
2   3  apple      0


In [10]:
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize

# 문장을 입력합니다.
sentence1 = "I love dogs"
sentence2 = "I adore canines"

# 문장을 단어로 분리합니다.
words1 = word_tokenize(sentence1)
words2 = word_tokenize(sentence2)

# 단어 유사도를 계산합니다.
similarity_scores = []
for word1 in words1:
    for word2 in words2:
        # 두 단어 각각의 동의어 세트를 구합니다.
        synsets1 = wordnet.synsets(word1)
        synsets2 = wordnet.synsets(word2)
        for synset1 in synsets1:
            for synset2 in synsets2:
                # 두 동의어 세트 간의 유사도를 계산합니다.
                similarity = synset1.path_similarity(synset2)
                if similarity is not None:
                    similarity_scores.append(similarity)

# 유사도 점수의 평균을 계산합니다.
if similarity_scores:
    average_similarity = sum(similarity_scores) / len(similarity_scores)
    print(f"The similarity score between '{sentence1}' and '{sentence2}' is {average_similarity:.2f}")
else:
    print("No similarity scores found.")

The similarity score between 'I love korea' and 'I adore canines' is 0.15


In [26]:
import pandas as pd
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize

# DataFrame 생성
df = pd.DataFrame({'id': [1, 2, 3], 'tag': ['monkey', 'cat', 'dog']})


# score 컬럼 추가
df['score'] = 0
user_input_word = 'ape'
user_input_words = word_tokenize(user_input_word)

# 단어 유사도 계산
for i, row in df.iterrows():
    tag = df.at[i,'tag']
    print(tag)
    sticker_words = word_tokenize(tag)
    similarity_scores = []
    
    for word1 in words1:
        for word2 in words2:
            # 두 단어 각각의 동의어 세트를 구합니다.
            synsets1 = wordnet.synsets(word1)
            synsets2 = wordnet.synsets(word2)
            for synset1 in synsets1:
                for synset2 in synsets2:
                    # 두 동의어 세트 간의 유사도를 계산합니다.
                    similarity = synset1.path_similarity(synset2)
                    if similarity is not None:
                        similarity_scores.append(similarity)
    
    if similarity_scores:
        average_similarity = sum(similarity_scores) / len(similarity_scores)
        print(f"The similarity score between '{user_input_word}' and '{tag}' is {average_similarity:.2f}")
    else:
        print("No similarity scores found.")
    

# 결과 출력
print(df)

monkey
The similarity score between 'ape' and 'monkey' is 0.15
cat
The similarity score between 'ape' and 'cat' is 0.15
dog
The similarity score between 'ape' and 'dog' is 0.15
   id     tag  score
0   1  monkey      0
1   2     cat      0
2   3     dog      0


In [21]:
from nltk.corpus import wordnet

# "big car"와 "huge vehicle" 각각의 단어에 대한 동의어 세트를 구합니다.
big_car_synsets = wordnet.synsets('big car')
huge_vehicle_synsets = wordnet.synsets('huge vehicle')

# 두 동의어 세트 간의 유사도를 계산합니다.
similarity_scores = []
for synset1 in big_car_synsets:
    for synset2 in huge_vehicle_synsets:
        similarity = synset1.path_similarity(synset2)
        if similarity is not None:
            similarity_scores.append(similarity)

# 유사도 점수의 평균을 계산합니다.
if similarity_scores:
    average_similarity = sum(similarity_scores) / len(similarity_scores)
    print(f"The average similarity score between 'big car' and 'huge vehicle' is {average_similarity:.2f}")
else:
    print("No similarity scores found.")

No similarity scores found.
